## Connect to Weaviate

> Use the same connection as you used in `1-setup`

### Connect to WCS

In [ ]:
import weaviate, os, json

client = weaviate.connect_to_wcs(
    cluster_url=os.getenv("WORKSHOP_DEMO_URL"),
    auth_credentials=weaviate.auth.AuthApiKey(os.getenv("WORKSHOP_DEMO_KEY_READ")),
    
    headers={
      "X-OpenAI-Api-Key": os.environ["OPENAI_API_KEY"] # replace with your own openai key
    }
)

client.is_ready()


### Connect with Docker

In [ ]:
import weaviate, os, json

# Connect to your local Weaviate instance
client = weaviate.connect_to_local(
    headers={
      "X-OpenAI-Api-Key": os.environ["OPENAI_API_KEY"]
    }
)

client.is_ready()

## Search

The `alpha` parameter determines the weight given to the sparse and dense search methods. `alpha = 0` is pure sparse (bm25) search, whereas `alpha = 1` is pure dense (vector) search. 

Alpha is an optional parameter. The default is set to `0.75`.

### Semantic Search

In [8]:
products = client.collections.get("Products")

response = products.query.near_text(
    # query="yoga top",
    # query="running shoes",
    # query="buty do biegania dla kobiet",
    query="trainers for girls",
    limit=5
)

for item in response.objects:
    print(item.properties["name"])
    # print(item.properties["sub_category"], "\n")
    # print(json.dumps(item.properties, indent=2))

Bulicorn Women's Cute Sports Running Shoes,Walking, Gym Casual Sneaker Lace-Up Shoes for Girl's
Nike Women's WMNS Gts '16 Txt Training and Running Shoes
MINARK Running, Walking, Gym, Training, Casual, Sports Shoes for Women
Adidas Womens Racer Tr21 Running Shoe
Adidas Girl's Hyperon W Running Shoes


### Keyword Search

In [11]:
import weaviate.classes.query as wq
products = client.collections.get("Products")

response = products.query.bm25(
    # query="running shoes",
    query="trainers for girls",
    query_properties=["name"],
    limit=5
)

for item in response.objects:
    print(item.properties["name"])

Bacca Bucci Impact Running Shoes/Trainers for Men for Gyming, Cross Training, Heavy Weight Lifting, Cross-fit with Natura...
Puma Girls Outerwear
Max Girls Leggings
Max Baby-Girls Legging
Hustle Bustle Always Weird Crop Hoodie fir Girls and Women Crop Sweatshirt and Crop Tops or Girls


### Hybrid Search

In [15]:
products = client.collections.get("Products")

response = products.query.hybrid(
    # query="yoga top",
    # query="running shoes",
    # query="running shoes for girls",
    query="trainers for girls",
    alpha=0.5,
    query_properties=["name"],
    limit=5,
)

for item in response.objects:
    print(item.properties["name"])

Bacca Bucci Impact Running Shoes/Trainers for Men for Gyming, Cross Training, Heavy Weight Lifting, Cross-fit with Natura...
Bulicorn Women's Cute Sports Running Shoes,Walking, Gym Casual Sneaker Lace-Up Shoes for Girl's
Nike Women's WMNS Gts '16 Txt Training and Running Shoes
MINARK Running, Walking, Gym, Training, Casual, Sports Shoes for Women
Adidas Womens Racer Tr21 Running Shoe


### Explain score

In [16]:
products = client.collections.get("Products")

response = products.query.hybrid(
    query="trainers for girls",
    alpha=0.5,
    query_properties=["name"],
    limit=5,
    return_metadata=wq.MetadataQuery(score=True, explain_score=True)
)

for item in response.objects:
    print(item.properties["name"])

    print(item.metadata.explain_score)
    print(item.metadata.score)
    print("=========")


Bacca Bucci Impact Running Shoes/Trainers for Men for Gyming, Cross Training, Heavy Weight Lifting, Cross-fit with Natura...

Hybrid (Result Set keyword) Document 6187ac08-ac14-413e-b5b4-0425c87b8684: original score 2.9932787, normalized score: 0.5
0.5
Bulicorn Women's Cute Sports Running Shoes,Walking, Gym Casual Sneaker Lace-Up Shoes for Girl's

Hybrid (Result Set vector) Document 66f7bdb5-5dd0-4309-b5cd-ae502b12bbdd: original score 0.8463192, normalized score: 0.5
0.5
Nike Women's WMNS Gts '16 Txt Training and Running Shoes

Hybrid (Result Set vector) Document 347a5778-3dd4-496d-b086-5f1654ff5747: original score 0.84567106, normalized score: 0.49073434
0.490734338760376
MINARK Running, Walking, Gym, Training, Casual, Sports Shoes for Women

Hybrid (Result Set vector) Document 2238aa59-921d-4091-a844-c64e034959bd: original score 0.84338725, normalized score: 0.4580855
0.4580855071544647
Adidas Womens Racer Tr21 Running Shoe

Hybrid (Result Set vector) Document 7e3b294a-53b3-451b-ab0b

### Fusion Type

In [17]:
import weaviate.classes.query as wq
response = products.query.hybrid(
    query="trainers for girls",
    query_properties=["name", "sub_category"],
    alpha=0.5,
    limit=5,
    fusion_type=wq.HybridFusion.RELATIVE_SCORE,
    return_metadata=wq.MetadataQuery(score=True, explain_score=True)
)

for item in response.objects:
    print(item.properties["name"])

    print(item.metadata.explain_score)
    print(item.metadata.score)
    print("=========")

Bacca Bucci Impact Running Shoes/Trainers for Men for Gyming, Cross Training, Heavy Weight Lifting, Cross-fit with Natura...

Hybrid (Result Set keyword) Document 6187ac08-ac14-413e-b5b4-0425c87b8684: original score 2.1123748, normalized score: 0.5
0.5
Bulicorn Women's Cute Sports Running Shoes,Walking, Gym Casual Sneaker Lace-Up Shoes for Girl's

Hybrid (Result Set vector) Document 66f7bdb5-5dd0-4309-b5cd-ae502b12bbdd: original score 0.84622574, normalized score: 0.5
0.5
Nike Women's WMNS Gts '16 Txt Training and Running Shoes

Hybrid (Result Set vector) Document 347a5778-3dd4-496d-b086-5f1654ff5747: original score 0.8456566, normalized score: 0.49186045
0.49186044931411743
MINARK Running, Walking, Gym, Training, Casual, Sports Shoes for Women

Hybrid (Result Set vector) Document 2238aa59-921d-4091-a844-c64e034959bd: original score 0.84333205, normalized score: 0.45861775
0.4586177468299866
KASGO Boys & Girls Training Shorts

Hybrid (Result Set vector) Document 691cf2f5-a4e4-45d8-aa84

In [18]:
import weaviate.classes.query as wq
response = products.query.hybrid(
    query="trainers for girls",
    query_properties=["name", "sub_category"],
    alpha=0.5,
    limit=5,
    fusion_type=wq.HybridFusion.RANKED,
    return_metadata=wq.MetadataQuery(score=True, explain_score=True)
)

for item in response.objects:
    print(item.properties["name"])

    print(item.metadata.explain_score)
    print(item.metadata.score)
    print("=========")

KASGO Boys & Girls Training Shorts

Hybrid (Result Set keyword) Document 691cf2f5-a4e4-45d8-aa84-06084b7663de contributed 0.007462686567164179 to the score
Hybrid (Result Set vector) Document 691cf2f5-a4e4-45d8-aa84-06084b7663de contributed 0.005555555555555556 to the score
0.013018242083489895
Qiddo Sports Girls' T-Shirt

Hybrid (Result Set keyword) Document 8cea5a2d-2979-4de3-b1b6-36c4c80cefbd contributed 0.0078125 to the score
Hybrid (Result Set vector) Document 8cea5a2d-2979-4de3-b1b6-36c4c80cefbd contributed 0.0036496350364963502 to the score
0.0114621352404356
Ckarfe Cycling Shorts for Women and Girls Yoga Shorts Gym Shorts

Hybrid (Result Set keyword) Document 7917a661-39fe-4833-a077-f22b3711d049 contributed 0.00510204081632653 to the score
Hybrid (Result Set vector) Document 7917a661-39fe-4833-a077-f22b3711d049 contributed 0.00625 to the score
0.01135204080492258
Girls Cotton Lycra Leggings Black

Hybrid (Result Set keyword) Document 8c9f6e10-6b8a-464c-a073-a38b65b8320f contrib

### Hybrid Search with a `where` filter

Find Jeopardy questions about elephants, where the category is set to Animals.

In [23]:
import weaviate.classes.query as wq

response = products.query.hybrid(
    query="running trousers",
    alpha=0.7,
    filters=wq.Filter.by_property("ratings").greater_than(4),
    limit=3
)

for item in response.objects:
    print(item.properties["name"])
    print(item.properties["ratings"])

hiker's way Track Pant for Men, Joggers for Men, Trackpants with Two Side Zipper Pocket for Sports Gym Athletic Training W...
4.1
FITINC Premium Track Pant for Men | Anti Microbial | Superdry | Breathable | Stretchable | 2 YKK Zipper Pockets
4.3
BLINKIN Women's 3/4 Gym Wear Tights for Women with Side Pockets : Perfect for Active Wear, Yoga & Workout - The Ultimate G...
4.1


## Generative Search

In [24]:
response = products.generate.hybrid(
    query="running trousers",
    alpha=0.5,
    filters=wq.Filter.by_property("ratings").greater_than(4),
    limit=3,
    single_prompt="Please give me a concise name for the following product, together with a one-paragraph description. {name}"
)

for item in response.objects:
    print("Original name:", item.properties["name"])
    print(item.generated)
    print("=======\n")

Original name: hiker's way Track Pant for Men, Joggers for Men, Trackpants with Two Side Zipper Pocket for Sports Gym Athletic Training W...
Product Name: Hiker's Way Men's Athletic Track Pants

Description: The Hiker's Way Men's Athletic Track Pants are designed for the active man, perfect for hiking, jogging, gym workouts, and athletic training. These versatile track pants feature two side zipper pockets for

Original name: FITINC Premium Track Pant for Men | Anti Microbial | Superdry | Breathable | Stretchable | 2 YKK Zipper Pockets
Product Name: FITINC Men's Premium Anti-Microbial Superdry Track Pants

Description: The FITINC Men's Premium Anti-Microbial Superdry Track Pants are designed for the modern, active man. These track pants are not only stylish but also functional, featuring an anti-m

Original name: BLINKIN Women's 3/4 Gym Wear Tights for Women with Side Pockets : Perfect for Active Wear, Yoga & Workout - The Ultimate G...
Product Name: BLINKIN Women's 3/4 Gym Tights with

In [25]:
response = products.generate.hybrid(
    query="running trousers",
    alpha=0.5,
    filters=wq.Filter.by_property("ratings").greater_than(4),
    limit=3,
    grouped_task="What do the provided products have in common? Please write a short advert I could post on LinkedIn.",
    grouped_properties=["name"]
)

print(response.generated)

All the products listed are athletic wear, specifically track pants and gym tights, designed for active lifestyles. They all feature convenient side zipper pockets and are made with materials that ensure comfort and flexibility during workouts or sports activities.

Advert: 
"Upgrade your workout wardrobe with our range of premium athletic wear! Whether you're hitting the gym, going for a run, or exploring the great outdoors, our track pants and gym tights are the perfect fit. Designed for both men and women, they feature convenient side zipper pockets to keep your essentials safe. Made with breathable, stretchable, and anti-microbial materials, they ensure maximum comfort and flexibility. Don't compromise on style
